# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

## Laplasjan z Gaussa (LoG)

Funkcja Gaussa:<br>
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}<br>
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
3. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
4. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
5. Wyświetl obraz wynikowy.
6. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os

if not os.path.exists("dom.png"):
    !wget https: // raw.githubusercontent.com / vision-agh / poc_sw / master / 09_Canny / dom.png --no-check-certificate

dom = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)


def gausssian_filter(img, thr, k_size):
    blur_img = cv2.GaussianBlur(img, k_size, 0)
    img_wy = crossing(cv2.Laplacian(blur_img, cv2.CV_32F), thr)

    plt.imshow(img_wy, "gray")
    plt.axis("off")
    plt.show()


def crossing(LoG, thr):
    tab_wy = np.zeros(LoG.shape)

    for i in range(1, LoG.shape[0] - 1):
        for j in range(1, LoG.shape[1] - 1):
            frame = LoG[i - 1: i + 1, i - 1: i + 1]
            max_value = np.max(frame)
            min_value = np.min(frame)
            tab_wy[i, j] = LoG[i, j]
            if np.sign(max_value) == -1 * np.sign(min_value):
                if tab_wy[i, j] > 0:
                    tab_wy[i, j] = tab_wy[i, j] + np.abs(min_value)
                if tab_wy[i, j] < 0:
                    tab_wy[i, j] = np.abs(tab_wy[i, j]) + max_value

    tab_wy = tab_wy / np.max(tab_wy) * 255
    bin_img = (tab_wy >= thr)
    tab_wy = (bin_img * 255).astype(np.uint8)
    tab_wy = cv2.medianBlur(tab_wy, 3)

    return tab_wy


gausssian_filter(dom, 37, (7, 7))

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w~1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

Zaimplementuj pierwszą część algorytmu detekcji krawędzi Canny'ego:
1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq TH $<br>
$g_{NL}(x,y) = TH > g_N(x,y) \geq TL $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
7. Na jednym obrazie zaznacz piksele należące do obrazu $g_{NH}$ jako czerwone oraz należące do obrazu $g_{NL}$ jako niebieskie.
Wyświetl obraz.

In [ ]:
def canny_edge_detection(img, t_low, t_high=None, k_size=(5, 5)):
    if t_high is None:
        t_high = 2.5 * t_low

    blur_img = cv2.GaussianBlur(img, k_size, 0)

    s1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    s2 = s1.T
    kernel_s1 = s1 / sum([el if el > 0 else 0 for row in s1 for el in row])
    kernel_s2 = s2 / sum([el if el > 0 else 0 for row in s2 for el in row])
    g_x = np.abs(cv2.filter2D(src=blur_img, ddepth=-1, kernel=kernel_s2))
    g_y = np.abs(cv2.filter2D(src=blur_img, ddepth=-1, kernel=kernel_s1))

    M = np.sqrt(g_x ** 2 + g_y ** 2)
    alfa = np.arctan2(g_y, g_x) / np.pi * 180  # stopnie
    alfa[alfa < 0] += 180

    g_n = np.zeros(M.shape)

    for i in range(1, M.shape[0] - 1):
        for j in range(1, M.shape[1] - 1):
            # frame = M[i - 1: i + 1, i - 1: i + 1]
            # kąt 0
            if 0 <= alfa[i, j] < 22.5 or 157.5 <= alfa[i, j] <= 180:
                if M[i, j - 1] > M[i, j] or M[i, j + 1] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]
            # kąt 45
            elif 22.5 <= alfa[i, j] < 67.5:
                if M[i + 1, j - 1] > M[i, j] or M[i - 1, j + 1] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]
            # kąt 90
            elif 67.5 <= alfa[i, j] < 112.5:
                if M[i + 1, j] > M[i, j] or M[i - 1, j] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]
            # kąt 135
            else:
                if M[i - 1, j - 1] > M[i, j] or M[i + 1, j + 1] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]

    gn_high = (g_n >= t_high)
    gn_low_1 = (g_n < t_high)
    gn_low_2 = (g_n >= t_low)
    gn_low = np.logical_and(gn_low_1, gn_low_2)

    img_wy = np.zeros((g_n.shape[0], g_n.shape[1], 3), dtype=np.uint8)
    img_wy[gn_low == True] = [10, 20, 220]
    img_wy[gn_high == True] = [250, 10, 0]

    return img_wy


edges = canny_edge_detection(dom, 5, 12.5)
plt.imshow(edges)
plt.axis('off')
plt.show()

## Algorytm Canny'ego - OpenCV

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z wykonaną częściową implementacją w poprzednim ćwiczeniu.
Na czym polegają różnice?

    Krawędzie są wyraźniejsze - są od siebie oddzielone, łatwo określić ich kierunek

In [ ]:
edges = cv2.Canny(dom, 5, 12.5, None, 3, 1)
plt.imshow(edges, 'gray')
plt.axis('off')
plt.show()